In [503]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import json
import os
import time
import random

# Generate permalinks

In [4]:
tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population')

In [15]:
cities = tables[2]

In [16]:
cities.columns = ['City', 'State', '2023 estimate', '2020 census', 'Change',
                 '2020 land area mi2', '2020 land area km2', '2020 density mi2',
                 '2020 density km2', 'Location']

In [17]:
cities = cities[['City', 'State']][:100]

In [19]:
cities['City'] = cities['City'].str.replace(r'\[.*?\]', '', regex=True).str.strip()


In [230]:
cities

,City,State,url
0,New York,NY,https://www.yelp.com/search?find_desc=Boutique...
1,Los Angeles,CA,https://www.yelp.com/search?find_desc=Boutique...
2,Chicago,IL,https://www.yelp.com/search?find_desc=Boutique...
3,Houston,TX,https://www.yelp.com/search?find_desc=Boutique...
4,Phoenix,AZ,https://www.yelp.com/search?find_desc=Boutique...
...,...,...,...
95,Norfolk,VA,https://www.yelp.com/search?find_desc=Boutique...
96,Spokane,WA,https://www.yelp.com/search?find_desc=Boutique...
97,Richmond,VA,https://www.yelp.com/search?find_desc=Boutique...
98,Fremont,CA,https://www.yelp.com/search?find_desc=Boutique...


In [501]:
cities.to_excel('cities.xlsx', index=False)

Example url: https://www.yelp.com/search?find_desc=Boutique+Shopping&find_loc=San+Francisco%2C+CA&start=0

In [21]:
for index, row in cities.iterrows():
    city = row['City']
    state = row['State']
    url = 'https://www.yelp.com/search?find_desc=Boutique+Shopping&find_loc=' + city.replace(' ', '+') + '%2C+' + state + '&start=0'
    cities.loc[index, 'url'] =  url

In [23]:
#cities.to_excel('cities.xlsx', index=False)

# Develop the scraper

In [314]:
folder_path = ''

In [398]:
# Initialize the DataFrame to store the scraped data
#scraped = pd.DataFrame(columns=['shop_name', 'shop_url', 'star_rating', 'reviews', 'category_1', 'category_2', 'category_3', 'City', 'State'])

In [393]:
url = "https://www.yelp.com/search"

querystring = {"find_desc":"Boutique Shopping","find_loc":"New York, NY",
              'start':'250'}

payload = ""
headers = {
    "cache-control": "max-age=0",
    "priority": "u=0, i",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"macOS\"",
    "sec-fetch-dest": "document",
    "sec-fetch-mode": "navigate",
    "sec-fetch-site": "same-origin",
    "sec-fetch-user": "?1",
    "upgrade-insecure-requests": "1",
}

response = requests.request("GET", url, data=payload, headers=headers, params=querystring)

print(response.text)

<!DOCTYPE html>
<html lang="en-US" prefix="og: http://ogp.me/ns#" style="margin: 0;padding: 0; border: 0; font-size: 100%; font: inherit; vertical-align: baseline;"><head><script>document.documentElement.className=document.documentElement.className.replace(/�no-js�/,"js");</script><meta http-equiv="Content-Language" content="en-US"><meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no"><link rel="mask-icon" sizes="any" href="https://s3-media0.fl.yelpcdn.com/assets/srv0/yelp_large_assets/b2bb2fb0ec9c/assets/img/logos/yelp_burst.svg" content="#FF1A1A"><link rel="shortcut icon" href="https://s3-media0.fl.yelpcdn.com/assets/srv0/yelp_large_assets/dcfe403147fc/assets/img/logos/favicon.ico"><script> window.ga=window.ga||function(){(ga.q=ga.q||[]).push(arguments)};ga.l=+new Date;window.ygaPageStartTime=new Date().getTime();</script><script>
            window.yelp = window.yelp || {};
            window.yelp.cookieTypePreferencesHeader = '["ANALYTICS","FUNCTIONA

In [394]:
response.status_code

200

In [395]:
soup = BeautifulSoup(response.text)

In [396]:
shops = soup.find_all('div', class_='y-css-1uij873')

In [397]:
for shop in shops:
    shop_name = shop.find('a', class_='y-css-12ly5yx')
    shop_url = shop.find('a', class_='y-css-12ly5yx')
    star_rating = shop.find('div', class_='y-css-9tnml4')
    reviews = shop.find('span', class_='y-css-wfbtsu')
    categories = shop.find_all('span', class_='y-css-1cn4gbs')

    # Extract text and attributes safely
    shop_name_text = shop_name.text if shop_name else None
    shop_url_href = shop_url['href'] if shop_url else None
    star_rating_aria = star_rating['aria-label'] if star_rating else None
    reviews_text = reviews.text if reviews else None

    # Extract categories safely
    category_1_text = categories[0].text if len(categories) > 0 else None
    category_2_text = categories[1].text if len(categories) > 1 else None
    category_3_text = categories[2].text if len(categories) > 2 else None

    print(shop_name_text, star_rating_aria, reviews_text, category_1_text, category_2_text, category_3_text)
    

In [409]:
cities[15:] # St. Louis

,City,State,url
15,Indianapolis,IN,https://www.yelp.com/search?find_desc=Boutique...
16,San Francisco,CA,https://www.yelp.com/search?find_desc=Boutique...
17,Seattle,WA,https://www.yelp.com/search?find_desc=Boutique...
18,Denver,CO,https://www.yelp.com/search?find_desc=Boutique...
19,Oklahoma City,OK,https://www.yelp.com/search?find_desc=Boutique...
...,...,...,...
95,Norfolk,VA,https://www.yelp.com/search?find_desc=Boutique...
96,Spokane,WA,https://www.yelp.com/search?find_desc=Boutique...
97,Richmond,VA,https://www.yelp.com/search?find_desc=Boutique...
98,Fremont,CA,https://www.yelp.com/search?find_desc=Boutique...


In [419]:
# Iterate over each city
for index, row in cities[88:].iterrows():
    city = row['City']
    state = row['State']
    start = 0
    
    while True:
        try:
            url = "https://www.yelp.com/search"
            querystring = {"find_desc": "Boutique Shopping", "find_loc": f"{city}, {state}", "start": str(start)}

            headers = {
                "cache-control": "max-age=0",
                "priority": "u=0, i",
                "sec-ch-ua-mobile": "?0",
                "sec-ch-ua-platform": "\"macOS\"",
                "sec-fetch-dest": "document",
                "sec-fetch-mode": "navigate",
                "sec-fetch-site": "same-origin",
                "sec-fetch-user": "?1",
                "upgrade-insecure-requests": "1",
            }

            response = requests.get(url, headers=headers, params=querystring)
            response.raise_for_status()
            time.sleep(5)
            soup = BeautifulSoup(response.text, 'html.parser')
            shops = soup.find_all('div', class_='y-css-1uij873')
            
            if not shops:
                # Break the loop if no shops are found
                break

            for shop in shops:
                shop_name = shop.find('a', class_='y-css-12ly5yx')
                shop_url = shop.find('a', class_='y-css-12ly5yx')
                star_rating = shop.find('div', class_='y-css-9tnml4')
                reviews = shop.find('span', class_='y-css-wfbtsu')
                categories = shop.find_all('span', class_='y-css-1cn4gbs')

                # Extract text and attributes safely
                shop_name_text = shop_name.text if shop_name else None
                shop_url_href = shop_url['href'] if shop_url else None
                star_rating_aria = star_rating['aria-label'] if star_rating else None
                reviews_text = reviews.text if reviews else None

                # Extract categories safely
                category_1_text = categories[0].text if len(categories) > 0 else None
                category_2_text = categories[1].text if len(categories) > 1 else None
                category_3_text = categories[2].text if len(categories) > 2 else None

                # Check if all extracted values are None
                if all(v is None for v in [shop_name_text, shop_url_href, star_rating_aria, reviews_text, category_1_text, category_2_text, category_3_text]):
                    break

                # Append to the DataFrame
                scraped = scraped.append({
                    'shop_name': shop_name_text,
                    'shop_url': shop_url_href,
                    'star_rating': star_rating_aria,
                    'reviews': reviews_text,
                    'category_1': category_1_text,
                    'category_2': category_2_text,
                    'category_3': category_3_text,
                    'City': city,
                    'State': state
                }, ignore_index=True)
                scraped.to_csv('scraped.csv', index=False)

            # Increment start for next page of results
            print('Done', index, city, start)
            start += 10

        except Exception as e:
            print(f'Exception: {e}')
            #break

Exception: HTTPSConnectionPool(host='www.yelp.com', port=443): Max retries exceeded with url: /search?find_desc=Boutique+Shopping&find_loc=Chesapeake%2C+VA&start=0 (Caused by ProxyError('Unable to connect to proxy', OSError('Tunnel connection failed: 408 Request Time-out')))
Done 88 Chesapeake 0
Done 88 Chesapeake 10
Done 88 Chesapeake 20
Done 88 Chesapeake 30
Done 88 Chesapeake 40
Done 88 Chesapeake 50
Done 88 Chesapeake 60
Done 88 Chesapeake 70
Done 88 Chesapeake 80
Done 88 Chesapeake 90
Done 88 Chesapeake 100
Done 88 Chesapeake 110
Done 88 Chesapeake 120
Done 88 Chesapeake 130
Exception: HTTPSConnectionPool(host='www.yelp.com', port=443): Max retries exceeded with url: /search?find_desc=Boutique+Shopping&find_loc=Chesapeake%2C+VA&start=140 (Caused by ProxyError('Unable to connect to proxy', OSError('Tunnel connection failed: 408 Request Time-out')))
Done 88 Chesapeake 140
Done 88 Chesapeake 150
Done 88 Chesapeake 160
Done 88 Chesapeake 170
Done 88 Chesapeake 180
Done 88 Chesapeake 1

In [420]:
len(scraped)

33196

In [442]:
scraped = scraped.drop_duplicates().reset_index(drop=True)

In [444]:
scraped = scraped.drop_duplicates(subset='shop_url')

In [446]:
# Drop duplicates within each city based on shop_name
scraped = scraped.groupby('City').apply(lambda x: x.drop_duplicates(subset='shop_name')).reset_index(drop=True)

In [448]:
scraped.dropna(subset=['star_rating'], inplace=True)

In [450]:
scraped['star_rating'] = scraped['star_rating'].apply(lambda x: float(x.replace(' star rating', '')))

In [452]:
scraped['reviews'] = scraped['reviews'].str.extract('(\d+\.?\d*k?)', expand=False).apply(convert_reviews)

In [454]:
scraped['reviews'].max()

2600

In [456]:
scraped = scraped[(scraped.category_1!='Shopping Centers')&(scraped.category_2!='Shopping Centers')&(scraped.category_3!='Shopping Centers')]

#Piercing
scraped = scraped[(scraped.category_1!='Piercing')&(scraped.category_2!='Piercing')&(scraped.category_3!='Piercing')]

In [457]:
scraped

,shop_name,shop_url,star_rating,reviews,category_1,category_2,category_3,City,State
0,Nordstrom Rack,/adredir?ad_business_id=0gHI7SWSED6Zmd0PzblhFg...,3.1,34,Accessories,Women's Clothing,Sports Wear,Albuquerque,NM
1,Vans,/adredir?ad_business_id=AU1dAS97-CX7sd9omMs_lQ...,3.0,2,Men's Clothing,Shoe Stores,Women's Clothing,Albuquerque,NM
3,The Wild Rose,/biz/the-wild-rose-albuquerque?osq=Boutique+Sh...,5.0,6,Women's Clothing,Accessories,None,Albuquerque,NM
4,Luna and Luz,/biz/luna-and-luz-albuquerque?osq=Boutique+Sho...,4.9,10,Jewelry,Accessories,Candle Stores,Albuquerque,NM
5,Strive,/biz/strive-albuquerque?osq=Boutique+Shopping,4.9,7,Women's Clothing,Accessories,Shoe Stores,Albuquerque,NM
...,...,...,...,...,...,...,...,...,...
14321,Soma,/biz/soma-winston-salem?osq=Boutique+Shopping,2.5,2,Women's Clothing,Lingerie,Swimwear,Winston-Salem,NC
14329,Hello Gorgeous,/biz/hello-gorgeous-lewisville-3?osq=Boutique+...,3.3,4,Women's Clothing,Accessories,Shoe Stores,Winston-Salem,NC
14330,Chico’s,/biz/chicos-winston-salem?osq=Boutique+Shopping,1.0,1,Women's Clothing,None,None,Winston-Salem,NC
14331,Slash Tags Boutique,/biz/slash-tags-boutique-winston-salem?osq=Bou...,5.0,1,"Used, Vintage & Consignment",None,None,Winston-Salem,NC


# Scraper 2

In [504]:
def sleep_with_random_delay(fixed_delay=5, random_range=(0.1, 1)):
    random_delay = random.uniform(*random_range)
    total_delay = fixed_delay + random_delay
    time.sleep(total_delay)

In [506]:
proxies = {
}


headers= {
}

response = requests.request(
    'GET',
    verify=False,
    proxies=proxies,
    headers=headers
)

print(response.text)

ProxyError: HTTPSConnectionPool(host='ip.smartproxy.com', port=443): Max retries exceeded with url: / (Caused by ProxyError('Unable to connect to proxy', NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7ff51ad6c9d0>: Failed to establish a new connection: [Errno 60] Operation timed out')))

In [502]:
folder_path = '/Users/anna.zemit/Desktop/Programming/2024/23. The Cities With the Most Loved Independent Stores/jsons/'

In [505]:
for index, row in cities[4:5].iterrows():
    city = row['City']
    state = row['State']
    start = 0

    while True:
        try:
            url = "https://www.yelp.com/search/snippet"

            querystring = {"find_desc":"Boutique Shopping",
                         "find_loc":f"{city}, {state}",
                         "start":str(start),
                         #"parent_request_id":"adfb45f5c248711a",
                         "request_origin":"user"}

            city_modified = city.replace(' ', '+')

            payload = ""
            headers = {
              #"accept": "*/*",
              #"accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
              #"cookie": "bse=1b031a5062ba4c2c8c624b253908f431; wdi=2|1A8F4537AFF6F5A8|0x1.99fe13b25fdd6p+30|1f8bf9ad4d7b983c; __adroll_fpc=d4f45228aa97c74c0e34216e6bd76251-1719633178199; _fbp=fb.1.1719633179934.837254864804109146; _gcl_au=1.1.892871009.1720928149; ndp_session_id=40fc78c0-a9b1-4a24-a712-7e243c16cec5; _scid=af82fb65-00b2-43e4-90ac-25daa965a513; _tt_enable_cookie=1; _ttp=9zyowBvv2ih1y1tuk7oo5paC4MK; _sctr=1%7C1720886400000; _ScCbts=%5B%5D; _ga=GA1.2.1A8F4537AFF6F5A8; hl=en_US; recentlocations=; location=%7B%22address1%22%3A+%22%22%2C+%22unformatted%22%3A+%22St.+Louis%22%2C+%22country%22%3A+%22US%22%2C+%22state%22%3A+%22MO%22%2C+%22address2%22%3A+%22%22%2C+%22city%22%3A+%22St.+Louis%22%2C+%22zip%22%3A+%22%22%2C+%22address3%22%3A+%22%22%7D; xcj=1|GIRRemht1V-wlHppABNbW7bulMcZXEwlIWXnZ8vqVw4; spses.d161=*; _ga_K9Z2ZEVC8C=GS1.2.1721045891.5.0.1721045891.0.0.0; _scid_r=af82fb65-00b2-43e4-90ac-25daa965a513; _uetsid=5447b72042a411efbf312dffb7a9f3bd; _uetvid=f1748d60536111eead8087f76e1e4e01; OptanonConsent=isGpcEnabled=0&datestamp=Mon+Jul+15+2024+20%3A18%3A27+GMT%2B0800+(China+Standard+Time)&version=202403.1.0&browserGpcFlag=0&isIABGlobal=false&identifierType=Cookie+Unique+Id&hosts=&consentId=676620ff-9d72-42f0-9193-1358364087d8&interactionCount=0&isAnonUser=1&landingPath=https%3A%2F%2Fwww.yelp.com%2Fsearch%3Ffind_desc%3DBoutique+Shopping%26find_loc%3DSan+Francisco%252C+CA%26start%3D10&groups=BG122%3A1%2CC0003%3A1%2CC0002%3A1%2CC0001%3A1%2CC0004%3A1; __ar_v4=7YX6SJQ4RZAMPB6LZ7CHFF%3A20240629%3A11%7CQB5JPFIKRZDSBOZSULG4YB%3A20240629%3A12%7CBHPKS4B4ONEJJMGH4QCJZR%3A20240629%3A12%7CCIUJFQIPMNEVFIW6EFR2RE%3A20240711%3A1; spid.d161=e76a9ab4-68af-4e04-9203-92cf8da07b22.1719633135.9.1721045969.1720963128.8c62eb1d-869b-4c81-8605-73d7ec53116e.27854ef8-49d7-4bc1-96b2-9d022af8f5a0.eca0c43b-4b09-41e5-a754-4a39aaf97e56.1721045889406.7; bsi=1%7C0b102ffa-99ca-4596-8b11-b06dd16a2cac%7C1721045969792%7C1721045884188",
              #"priority": "u=1, i",
              "referer": f"https://www.yelp.com/search?find_desc=Boutique+Shopping&find_loc={city_modified}%2C+{state}&start={str(start)}",
              #"sec-ch-ua": "\"Not/A)Brand\";v=\"8\", \"Chromium\";v=\"126\", \"Google Chrome\";v=\"126\"",
              #"sec-ch-ua-mobile": "?0",
              #"sec-ch-ua-platform": "\"macOS\"",
              #"sec-fetch-dest": "empty",
              #"sec-fetch-mode": "cors",
              #"sec-fetch-site": "same-origin",
              "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36"
          }

            response = requests.request("GET", url, data=payload, headers=headers, params=querystring, verify=False, proxies=proxies)
            response.raise_for_status()
            sleep_with_random_delay()

            response_dict = response.json()

            exception = response_dict['searchPageProps'].get('searchExceptionProps', None)

            if pd.notna(exception):
                break

          # Save the response JSON to a file
            file_name = f'{city}, {state}_{start}.json'
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'w') as json_file:
                json.dump(response_dict, json_file, indent=4)

            print('Done', index, city, start)
            start += 10

        except Exception as e:
            print(f'Exception: {e}')
            break


Exception: Unterminated string starting at: line 1 column 115708 (char 115707)


# Read scraped data

In [460]:
data_1 = pd.read_csv('scraped_data_octo.csv')
data_2 = pd.read_csv('scraped_data_octo(1).csv')
data_3 = pd.read_csv('scraped_data_octo(2).csv')
data_4 = pd.read_csv('scraped_data_octo_1.csv')
data_5 = pd.read_csv('scraped_data_octo_1(1).csv')


data = pd.concat([data_1, data_2, data_3, data_4, data_5])

In [461]:
data = data.drop_duplicates().reset_index(drop=True)

In [462]:
data = data.merge(cities, on='url', how='left')

In [463]:
data = data.drop_duplicates(subset='shop_url')

In [464]:
data['shop_name'] = data['shop_name'].str.replace(r'\d+\.\s', '')

/Users/anna.zemit/opt/anaconda3/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [477]:
# Drop duplicates within each city based on shop_name
data = data.groupby('City').apply(lambda x: x.drop_duplicates(subset='shop_name')).reset_index(drop=True)

In [485]:
city_counts = scraped['City'].value_counts(ascending=True).reset_index()

# Rename the columns for better understanding
city_counts.columns = ['City', 'Count']

In [486]:
city_counts #.to_excel('city_counts.xlsx')

,City,Count
0,Laredo,5
1,Norfolk,8
2,Chesapeake,13
3,North Las Vegas,15
4,St. Louis,16
...,...,...
93,Portland,222
94,San Francisco,228
95,New York,229
96,Chicago,230


In [251]:
#data = data.drop_duplicates(subset='shop_name')

In [466]:
data.dropna(subset=['star_rating'], inplace=True)

In [467]:
data = data[['shop_name', 'shop_url', 'City', 'State', 'star_rating', 'reviews', 
             'category_1','category_2', 'category_3',]]

In [468]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12201 entries, 0 to 15096
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   shop_name    12201 non-null  object 
 1   shop_url     12201 non-null  object 
 2   City         12201 non-null  object 
 3   State        12201 non-null  object 
 4   star_rating  12201 non-null  float64
 5   reviews      12201 non-null  object 
 6   category_1   12201 non-null  object 
 7   category_2   9209 non-null   object 
 8   category_3   5261 non-null   object 
dtypes: float64(1), object(8)
memory usage: 953.2+ KB


In [433]:
# Define a function to convert 'k' to thousands
def convert_reviews(reviews_str):
    if 'k' in reviews_str:
        return int(float(reviews_str.replace('k', '')) * 1000)
    return int(reviews_str)

In [469]:
data['reviews'] = data['reviews'].str.extract('(\d+\.?\d*k?)', expand=False).apply(convert_reviews)

In [470]:
data['reviews'].max()

2600

In [471]:
len(data.City.unique())

100

In [472]:
data = data[(data.category_1!='Shopping Centers')&(data.category_2!='Shopping Centers')&(data.category_3!='Shopping Centers')]

In [473]:
#Piercing
data = data[(data.category_1!='Piercing')&(data.category_2!='Piercing')&(data.category_3!='Piercing')]

In [475]:
data = pd.concat([data, scraped])

In [553]:
data_old = data.copy()

In [554]:
data_old

,shop_name,shop_url,City,State,star_rating,reviews,category_1,category_2,category_3,weighted_rating
0,Vans,https://www.yelp.com/adredir?ad_business_id=AU...,Albuquerque,NM,3.0,2,Shoe Stores,Men's Clothing,Women's Clothing,6.0
1,Nordstrom Rack,https://www.yelp.com/adredir?ad_business_id=0g...,Albuquerque,NM,3.1,34,Sports Wear,Men's Clothing,Shoe Stores,105.4
2,Macy’s,https://www.yelp.com/adredir?ad_business_id=mt...,Albuquerque,NM,3.4,28,Bridal,Department Stores,Men's Clothing,95.2
3,The Wild Rose,https://www.yelp.com/biz/the-wild-rose-albuque...,Albuquerque,NM,5.0,6,Women's Clothing,Accessories,NaN,30.0
4,Luna and Luz,https://www.yelp.com/biz/luna-and-luz-albuquer...,Albuquerque,NM,4.9,10,Jewelry,Accessories,Candle Stores,49.0
...,...,...,...,...,...,...,...,...,...,...
12327,Nitsa’s Apparel,https://www.yelp.com/biz/nitsas-apparel-winsto...,Winston-Salem,NC,4.4,14,Bridal,Sewing & Alterations,NaN,61.6
12328,Shoe Carnival,https://www.yelp.com/adredir?ad_business_id=fo...,Winston-Salem,NC,2.8,9,Accessories,Shoe Stores,NaN,25.2
12329,Soma,https://www.yelp.com/biz/soma-winston-salem?os...,Winston-Salem,NC,2.5,2,Women's Clothing,Lingerie,NaN,5.0
12330,Gaia,/biz/gaia-winston-salem?osq=Boutique+Shopping,Winston-Salem,NC,5.0,5,Women's Clothing,Accessories,None,25.0


In [497]:
#data.to_excel('raw_data.xlsx')

## Scraped jsons

In [647]:
folder_path = '/Users/anna.zemit/Desktop/Programming/2024/23. The Cities With the Most Loved Independent Stores/jsons/'

json_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]

In [689]:
data = pd.DataFrame(columns=['shop_name', 'shop_url', 'star_rating', 'reviews', 'category_1', 'category_2', 'category_3', 'City', 'State'])

In [690]:
for file in json_files:
    with open(os.path.join(folder_path, file), 'r') as json_file:
        json_data = json.load(json_file)
        file_name = file.split('.json')[0]
        city = file_name.split(', ')[0]
        state = file_name.split(', ')[1].split('_')[0]
        
        shops = json_data['searchPageProps']['mainContentComponentsListProps']
        
        for shop in shops:
            searchResultLayoutType = shop.get('searchResultLayoutType', None)
            
            if searchResultLayoutType == 'iaResult':
                name = shop['searchResultBusiness']['name']
                businessUrl = shop['searchResultBusiness']['businessUrl']
                categories = shop['searchResultBusiness']['categories']
                
                bizId = shop['bizId']
                
                category_1 = categories[0]['title'] if len(categories) > 0 else None
                category_2 = categories[1]['title'] if len(categories) > 1 else None
                category_3 = categories[2]['title'] if len(categories) > 2 else None
                
                rating = shop['searchResultBusiness']['rating']
                reviewCount = shop['searchResultBusiness']['reviewCount']
                
                # Append to the DataFrame
                data = data.append({
                    'shop_name': name,
                    #'business_id': bizId,
                    'shop_url': businessUrl,
                    'star_rating': rating,
                    'reviews': reviewCount,
                    'category_1': category_1,
                    'category_2': category_2,
                    'category_3': category_3,
                    'City': city,
                    'State': state
                }, ignore_index=True)
                

In [691]:
len(data)

34410

In [692]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34410 entries, 0 to 34409
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   shop_name    34410 non-null  object 
 1   shop_url     34410 non-null  object 
 2   star_rating  34410 non-null  float64
 3   reviews      34410 non-null  object 
 4   category_1   34410 non-null  object 
 5   category_2   29301 non-null  object 
 6   category_3   21675 non-null  object 
 7   City         34410 non-null  object 
 8   State        34410 non-null  object 
dtypes: float64(1), object(8)
memory usage: 2.4+ MB


In [693]:
data = data.drop_duplicates()

In [694]:
# Drop duplicates within each city based on shop_name
data = data.groupby('City').apply(lambda x: x.drop_duplicates(subset='shop_name')).reset_index(drop=True)

In [695]:
data = data[(data.category_1!='Shopping Centers')&(data.category_2!='Shopping Centers')&(data.category_3!='Shopping Centers')]

#Piercing
data = data[(data.category_1!='Piercing')&(data.category_2!='Piercing')&(data.category_3!='Piercing')]

In [696]:
data = data[~((data.star_rating == 0) & (data.reviews != 0))]

In [697]:
data

,shop_name,shop_url,star_rating,reviews,category_1,category_2,category_3,City,State
0,Nordstrom Rack,/adredir?ad_business_id=0gHI7SWSED6Zmd0PzblhFg...,3.1,34,Accessories,Shoe Stores,Discount Store,Albuquerque,NM
1,Vans,/adredir?ad_business_id=AU1dAS97-CX7sd9omMs_lQ...,3.0,2,Men's Clothing,Shoe Stores,Women's Clothing,Albuquerque,NM
3,Sparkle,/biz/sparkle-los-lunas?osq=Boutique+Shopping,5.0,1,Jewelry,None,None,Albuquerque,NM
4,Grace Thrift Store,/biz/grace-thrift-store-albuquerque?osq=Boutiq...,3.0,2,Thrift Stores,None,None,Albuquerque,NM
5,Talking Talons Thrift Store,/biz/talking-talons-thrift-store-cedar-crest?o...,2.7,10,Thrift Stores,None,None,Albuquerque,NM
...,...,...,...,...,...,...,...,...,...
18388,McCalls Reynolda Village,/biz/mccalls-reynolda-village-winston-salem?os...,5.0,1,Lingerie,Children's Clothing,Shoe Stores,Winston-Salem,NC
18389,Fashion Over All,/biz/fashion-over-all-winston-salem?osq=Boutiq...,0.0,0,Women's Clothing,Accessories,Shoe Stores,Winston-Salem,NC
18390,Pink Culture Mobile Boutique,/biz/pink-culture-mobile-boutique-winston-sale...,0.0,0,Women's Clothing,None,None,Winston-Salem,NC
18391,Keona’s Boutique,/biz/keonas-boutique-winston-salem?osq=Boutiqu...,0.0,0,Women's Clothing,Accessories,None,Winston-Salem,NC


## Add data

In [666]:
data_add = pd.DataFrame(columns=['business_id', 'star_rating', 'reviews',])

In [667]:
for file in json_files:
    with open(os.path.join(folder_path, file), 'r') as json_file:
        json_data = json.load(json_file)
        file_name = file.split('.json')[0]
        city = file_name.split(', ')[0]
        state = file_name.split(', ')[1].split('_')[0]
        
        dict_keys = json_data['searchPageProps']['rightRailProps']['searchMapProps']['hovercardData'].keys()
        
        for key in dict_keys:
            shop_data = json_data['searchPageProps']['rightRailProps']['searchMapProps']['hovercardData'][key]
            
            bizId = shop_data.get('bizId', None)
            rating = shop_data.get('rating', None)
            numReviews = shop_data.get('numReviews', None)
            
            # Append to the DataFrame
            data_add = data_add.append({
                    'business_id': bizId,
                    'star_rating': rating,
                    'reviews': numReviews,
                }, ignore_index=True)
            

In [670]:
data_add = data_add.drop_duplicates()

In [673]:
data_add

,business_id,star_rating,reviews
0,FpLIEaRSdOVf3RnZiPkg9Q,0.0,0
1,GnrMaZw_ITiWubNbL7ejug,4.1,49
2,kjNcsl7SHgvOhIkECb0s3g,3.2,5
3,5KHD3mKWqJtbR-V89wsGWA,5.0,1
4,O5da_LTEMlNwlCnQFI7IpQ,3.3,7
...,...,...,...
34400,3LJgsQFCYdDK2bFWhdAcVg,4.8,10
34401,Roy9pImEYoEO6hNIOmG3Lw,0.0,0
34402,bMD3ipKFyPCqMzyYigBIqw,4.3,24
34403,i1l_PATwxBBBHVhCRXI3Tw,3.7,3


# Analysis

## Calculate the weighted average rating for boutique stores in each city

Calculate average rating based on the individual store ratings and number of reviews.


In [698]:
# Calculate the weighted sum of ratings for each city
data['weighted_rating'] = data['star_rating'] * data['reviews']

In [699]:
# Group by City and calculate the weighted average rating
weighted_avg_rating = data.groupby('City').apply(
    lambda x: x['weighted_rating'].sum() / x['reviews'].sum()
).reset_index(name='weighted_avg_rating')


In [700]:
store_counts = data.groupby('City').size().reset_index(name='stores_number')


In [701]:
weighted_avg_rating = weighted_avg_rating.merge(store_counts, on='City')

In [702]:
weighted_avg_rating = weighted_avg_rating.merge(cities[['City', 'State']], on='City')

In [703]:
weighted_avg_rating = weighted_avg_rating[['City', 'State', 'weighted_avg_rating', 'stores_number',]]

In [704]:
weighted_avg_rating.stores_number.describe()

count    100.00000
mean     174.41000
std       69.69531
min       21.00000
25%      114.25000
50%      218.50000
75%      228.00000
max      243.00000
Name: stores_number, dtype: float64

In [705]:
weighted_avg_rating

,City,State,weighted_avg_rating,stores_number
0,Albuquerque,NM,3.808602,72
1,Anaheim,CA,4.091966,236
2,Anchorage,AK,3.812471,43
3,Arlington,TX,3.456133,139
4,Atlanta,GA,3.889469,221
...,...,...,...,...
95,Tulsa,OK,4.009270,88
96,Virginia Beach,VA,3.761712,156
97,Washington,DC,3.801503,225
98,Wichita,KS,3.669167,46


## Calculate weighted average rating for each category of boutique store in each city

Calculate the weighted average rating for stores of each category with at least 10 stores in each city (e.g. women’s clothing, accessories).


In [706]:
# Melt the DataFrame to have a single category column
melted_df = data.melt(id_vars=['shop_name', 'shop_url', 'City', 'State', 'star_rating', 'reviews'], 
                    value_vars=['category_1', 'category_2', 'category_3'],
                    var_name='category_type', value_name='category')


In [707]:
melted_df

,shop_name,shop_url,City,State,star_rating,reviews,category_type,category
0,Nordstrom Rack,/adredir?ad_business_id=0gHI7SWSED6Zmd0PzblhFg...,Albuquerque,NM,3.1,34,category_1,Accessories
1,Vans,/adredir?ad_business_id=AU1dAS97-CX7sd9omMs_lQ...,Albuquerque,NM,3.0,2,category_1,Men's Clothing
2,Sparkle,/biz/sparkle-los-lunas?osq=Boutique+Shopping,Albuquerque,NM,5.0,1,category_1,Jewelry
3,Grace Thrift Store,/biz/grace-thrift-store-albuquerque?osq=Boutiq...,Albuquerque,NM,3.0,2,category_1,Thrift Stores
4,Talking Talons Thrift Store,/biz/talking-talons-thrift-store-cedar-crest?o...,Albuquerque,NM,2.7,10,category_1,Thrift Stores
...,...,...,...,...,...,...,...,...
52318,McCalls Reynolda Village,/biz/mccalls-reynolda-village-winston-salem?os...,Winston-Salem,NC,5.0,1,category_3,Shoe Stores
52319,Fashion Over All,/biz/fashion-over-all-winston-salem?osq=Boutiq...,Winston-Salem,NC,0.0,0,category_3,Shoe Stores
52320,Pink Culture Mobile Boutique,/biz/pink-culture-mobile-boutique-winston-sale...,Winston-Salem,NC,0.0,0,category_3,None
52321,Keona’s Boutique,/biz/keonas-boutique-winston-salem?osq=Boutiqu...,Winston-Salem,NC,0.0,0,category_3,None


In [708]:
# Group by City and Category to count the number of stores in each category per city
category_counts = melted_df.groupby(['City', 'category']).size().reset_index(name='store_count')
category_counts

,City,category,store_count
0,Albuquerque,Accessories,26
1,Albuquerque,Antiques,1
2,Albuquerque,Art Galleries,1
3,Albuquerque,Bridal,2
4,Albuquerque,Candle Stores,1
...,...,...,...
4996,Winston-Salem,Thrift Stores,1
4997,Winston-Salem,Toy Stores,1
4998,Winston-Salem,"Used, Vintage & Consignment",4
4999,Winston-Salem,Watches,2


In [709]:
# Filter out categories with fewer than 10 stores in each city
valid_categories = category_counts[category_counts['store_count'] >= 10]
valid_categories

,City,category,store_count
0,Albuquerque,Accessories,26
18,Albuquerque,Jewelry,17
29,Albuquerque,Thrift Stores,12
30,Albuquerque,"Used, Vintage & Consignment",12
32,Albuquerque,Women's Clothing,36
...,...,...,...
4972,Wichita,Women's Clothing,30
4973,Winston-Salem,Accessories,27
4985,Winston-Salem,Jewelry,11
4993,Winston-Salem,Shoe Stores,10


In [710]:
# Merge the valid categories back with the melted DataFrame
filtered_df = pd.merge(melted_df, valid_categories[['City', 'category']], on=['City', 'category'])
filtered_df

,shop_name,shop_url,City,State,star_rating,reviews,category_type,category
0,Nordstrom Rack,/adredir?ad_business_id=0gHI7SWSED6Zmd0PzblhFg...,Albuquerque,NM,3.1,34,category_1,Accessories
1,Two J’s Boutique,/biz/two-js-boutique-bosque-farms?osq=Boutique...,Albuquerque,NM,0.0,0,category_1,Accessories
2,Pink Rhino,/biz/pink-rhino-albuquerque-3?osq=Boutique+Sho...,Albuquerque,NM,2.8,5,category_1,Accessories
3,Retrohex,/biz/retrohex-albuquerque?osq=Boutique+Shopping,Albuquerque,NM,5.0,1,category_1,Accessories
4,The Wild Rose,/biz/the-wild-rose-albuquerque?osq=Boutique+Sh...,Albuquerque,NM,5.0,6,category_2,Accessories
...,...,...,...,...,...,...,...,...
30481,Umami Home Gift,/biz/umami-home-gift-albany-4?osq=Boutique+Sho...,Oakland,CA,5.0,27,category_3,Gift Shops
30482,Modern Mouse,/biz/modern-mouse-alameda-2?osq=Boutique+Shopping,Oakland,CA,4.6,104,category_3,Gift Shops
30483,Marisa Mason,/biz/marisa-mason-berkeley?osq=Boutique+Shopping,Oakland,CA,4.6,35,category_3,Gift Shops
30484,Wardrobe Boutique,/biz/wardrobe-boutique-walnut-creek?osq=Boutiq...,Oakland,CA,5.0,14,category_3,Gift Shops


In [711]:
# Calculate the weighted average rating for each category in each city
filtered_df['weighted_rating'] = filtered_df['star_rating'] * filtered_df['reviews']
weighted_avg_categories = filtered_df.groupby(['City', 'category']).apply(
    lambda x: x['weighted_rating'].sum() / x['reviews'].sum()
).reset_index(name='weighted_avg_rating')


In [712]:
# Add the number of stores to the weighted average rating DataFrame
weighted_avg_categories = pd.merge(weighted_avg_categories, category_counts, on=['City', 'category'])
weighted_avg_categories

,City,category,weighted_avg_rating,store_count
0,Albuquerque,Accessories,4.146181,26
1,Albuquerque,Jewelry,4.306875,17
2,Albuquerque,Thrift Stores,3.323267,12
3,Albuquerque,"Used, Vintage & Consignment",3.515385,12
4,Albuquerque,Women's Clothing,3.686572,36
...,...,...,...,...
779,Wichita,Women's Clothing,3.600826,30
780,Winston-Salem,Accessories,3.678261,27
781,Winston-Salem,Jewelry,4.450000,11
782,Winston-Salem,Shoe Stores,3.986207,10


In [713]:
weighted_avg_categories = weighted_avg_categories.merge(cities[['City', 'State']], on='City')

weighted_avg_categories = weighted_avg_categories[['City', 'State', 'category', 'weighted_avg_rating', 'store_count',]]

In [714]:
weighted_avg_categories

,City,State,category,weighted_avg_rating,store_count
0,Albuquerque,NM,Accessories,4.146181,26
1,Albuquerque,NM,Jewelry,4.306875,17
2,Albuquerque,NM,Thrift Stores,3.323267,12
3,Albuquerque,NM,"Used, Vintage & Consignment",3.515385,12
4,Albuquerque,NM,Women's Clothing,3.686572,36
...,...,...,...,...,...
779,Wichita,KS,Women's Clothing,3.600826,30
780,Winston-Salem,NC,Accessories,3.678261,27
781,Winston-Salem,NC,Jewelry,4.450000,11
782,Winston-Salem,NC,Shoe Stores,3.986207,10


## Refer data to Barnes

In [715]:
# Create an Excel writer object

writer = pd.ExcelWriter('The Cities With the Most Loved Independent Stores Pick data.xlsx')

In [716]:
# Save each DataFrame to a separate sheet in the Excel file

weighted_avg_rating.to_excel(writer, sheet_name='Cities', index=True)
weighted_avg_categories.to_excel(writer, sheet_name='Categories', index=True)
data.to_excel(writer, sheet_name='Raw Data', index=True)

In [717]:
# Save the Excel file
writer.save()

## Isolate the highest rated stores

### Top 20 overall

In [718]:
top_20 = data[(data.category_1!='Shopping Centers')&(data.category_2!='Shopping Centers')&(data.category_3!='Shopping Centers')]

In [719]:
top_20 = top_20.sort_values(by='weighted_rating', ascending=False).head(20)

In [720]:
top_20

,shop_name,shop_url,star_rating,reviews,category_1,category_2,category_3,City,State,weighted_rating
118,Happy Jewelers,/biz/happy-jewelers-fullerton-2?osq=Boutique+S...,4.9,1623,Jewelry,Watches,Jewelry Repair,Anaheim,CA,7952.7
15746,Trudys Brides & Special Occasions,/adredir?ad_business_id=TAzqX8RQr1GvPYnqgZwGZQ...,3.9,1815,Women's Clothing,Bridal,None,San Jose,CA,7078.5
16185,Tustin Jewelry Exchange - Direct Diamond Importer,/biz/tustin-jewelry-exchange-direct-diamond-im...,4.4,1440,Jewelry,Jewelry Repair,None,Santa Ana,CA,6336.0
15272,Road Runner Sports,/biz/road-runner-sports-san-diego?osq=Boutique...,3.7,1574,Sports Wear,Shoe Stores,None,San Diego,CA,5823.8
5242,Janene’s Bridal Boutique,/biz/janenes-bridal-boutique-alameda?osq=Bouti...,4.5,1099,Bridal,Accessories,Sewing & Alterations,Fremont,CA,4945.5
11923,Janene’s Bridal Boutique,/biz/janenes-bridal-boutique-alameda?osq=Bouti...,4.5,1097,Bridal,Accessories,Sewing & Alterations,Oakland,CA,4936.5
2939,The Diamond Connection,/biz/the-diamond-connection-san-diego-2?osq=Bo...,4.9,929,Jewelry,Diamond Buyers,None,Chula Vista,CA,4552.1
2954,David’s Bridal,/biz/davids-bridal-san-diego-5?osq=Boutique+Sh...,3.6,1210,Bridal,Accessories,Shoe Stores,Chula Vista,CA,4356.0
6630,Don Quijote - Kaheka,/biz/don-quijote-kaheka-honolulu-2?osq=Boutiqu...,4.0,1084,Grocery,Department Stores,International Grocery,Honolulu,HI,4336.0
5062,Sports Basement,/biz/sports-basement-sunnyvale-2?osq=Boutique+...,4.2,975,Sports Wear,Bikes,Outdoor Gear,Fremont,CA,4095.0


### Top store for winning cities overall

In [721]:
# Convert 'weighted_rating' to numeric, forcing errors to NaN
data['weighted_rating'] = pd.to_numeric(data['weighted_rating'], errors='coerce')

# Check for any conversion errors
if data['weighted_rating'].isnull().any():
    print("There are non-numeric values in 'weighted_rating' that could not be converted.")

# Now apply the groupby and idxmax
idx = data.groupby('City')['weighted_rating'].idxmax()

# Select the rows corresponding to these indices
top_stores = data.loc[idx]

In [722]:
top_stores = top_stores[['shop_name', 'shop_url', 'City']]

In [723]:
top_stores.columns = ['top_shop', 'shop_url', 'City']

In [724]:
top_stores

,top_shop,shop_url,City
63,Buffalo Exchange,/biz/buffalo-exchange-albuquerque?osq=Boutique...,Albuquerque
118,Happy Jewelers,/biz/happy-jewelers-fullerton-2?osq=Boutique+S...,Anaheim
367,REI,/biz/rei-anchorage-2?osq=Boutique+Shopping,Anchorage
419,Stardust Celebrations,/biz/stardust-celebrations-plano?osq=Boutique+...,Arlington
723,Worthmore Jewelers,/biz/worthmore-jewelers-decatur?osq=Boutique+S...,Atlanta
...,...,...,...
17831,Ida Red,/biz/ida-red-tulsa-2?osq=Boutique+Shopping,Tulsa
18003,Uptown Cheapskate - Virginia Beach,/biz/uptown-cheapskate-virginia-beach-virginia...,Virginia Beach
18258,David’s Bridal,/biz/davids-bridal-springfield-14?osq=Boutique...,Washington
18291,Lucinda’s,/biz/lucindas-wichita?osq=Boutique+Shopping,Wichita


### Top stores for winning cities in focus categories

#### Jewelry

In [725]:
idx = data[(data.category_1=='Jewelry')|(data.category_2=='Jewelry')|(data.category_3=='Jewelry')].groupby('City')['weighted_rating'].idxmax()

In [726]:
top_stores_jewelry = data.loc[idx]

In [727]:
top_stores_jewelry = top_stores_jewelry[['shop_name', 'shop_url', 'City']]

top_stores_jewelry.columns = ['top_shop', 'shop_url', 'City']

In [728]:
top_stores_jewelry

,top_shop,shop_url,City
69,Sukhmani Jewelry,/biz/sukhmani-jewelry-albuquerque?osq=Boutique...,Albuquerque
118,Happy Jewelers,/biz/happy-jewelers-fullerton-2?osq=Boutique+S...,Anaheim
336,Orainda’s Bridal Boutique,/biz/oraindas-bridal-boutique-anchorage?osq=Bo...,Anchorage
457,Hank & Ellie,/biz/hank-and-ellie-pantego?osq=Boutique+Shopping,Arlington
723,Worthmore Jewelers,/biz/worthmore-jewelers-decatur?osq=Boutique+S...,Atlanta
...,...,...,...
17806,Spexton Jewelry,/biz/spexton-jewelry-tulsa?osq=Boutique+Shopping,Tulsa
17938,Adele Diamond,/biz/adele-diamond-virginia-beach-2?osq=Boutiq...,Virginia Beach
18114,Tiny Jewel Box,/biz/tiny-jewel-box-washington?osq=Boutique+Sh...,Washington
18291,Lucinda’s,/biz/lucindas-wichita?osq=Boutique+Shopping,Wichita


#### Accessories

In [729]:
idx = data[(data.category_1=='Accessories')|(data.category_2=='Accessories')|(data.category_3=='Accessories')].groupby('City')['weighted_rating'].idxmax()

top_stores_accessories = data.loc[idx]

top_stores_accessories = top_stores_accessories[['shop_name', 'shop_url', 'City']]

top_stores_accessories.columns = ['top_shop', 'shop_url', 'City']

top_stores_accessories

,top_shop,shop_url,City
43,Heart & Sole Sports,/biz/heart-and-sole-sports-albuquerque?osq=Bou...,Albuquerque
107,Kaitlyn Clothing,/biz/kaitlyn-clothing-brea-4?osq=Boutique+Shop...,Anaheim
365,ShuzyQ,/biz/shuzyq-anchorage?osq=Boutique+Shopping,Anchorage
495,Uptown Cheapskate - Colleyville,/biz/uptown-cheapskate-colleyville-euless?osq=...,Arlington
551,The Beehive,/biz/the-beehive-atlanta?osq=Boutique+Shopping,Atlanta
...,...,...,...
17787,Nordstrom Rack,/adredir?ad_business_id=tUWct9zouwTNGdVJfPPEGg...,Tulsa
17956,Anderson’s - Newport News,/biz/andersons-newport-news-newport-news?osq=B...,Virginia Beach
18052,Le Bustiere Boutique,/adredir?ad_business_id=ffODXJGgk-7pkBITmaNx7Q...,Washington
18312,David’s Bridal,/biz/davids-bridal-wichita-3?osq=Boutique+Shop...,Wichita


#### Women's Clothing

In [730]:
idx = data[(data.category_1=='Women\'s Clothing')|(data.category_2=='Women\'s Clothing')|(data.category_3=='Women\'s Clothing')].groupby('City')['weighted_rating'].idxmax()

top_stores_women_clothing = data.loc[idx]

top_stores_women_clothing = top_stores_women_clothing[['shop_name', 'shop_url', 'City']]

top_stores_women_clothing.columns = ['top_shop', 'shop_url', 'City']

top_stores_women_clothing

,top_shop,shop_url,City
63,Buffalo Exchange,/biz/buffalo-exchange-albuquerque?osq=Boutique...,Albuquerque
107,Kaitlyn Clothing,/biz/kaitlyn-clothing-brea-4?osq=Boutique+Shop...,Anaheim
365,ShuzyQ,/biz/shuzyq-anchorage?osq=Boutique+Shopping,Anchorage
377,Whole Earth Provision,/adredir?ad_business_id=QN_Jz9kYklpE5o3fheAB_g...,Arlington
551,The Beehive,/biz/the-beehive-atlanta?osq=Boutique+Shopping,Atlanta
...,...,...,...
17831,Ida Red,/biz/ida-red-tulsa-2?osq=Boutique+Shopping,Tulsa
18003,Uptown Cheapskate - Virginia Beach,/biz/uptown-cheapskate-virginia-beach-virginia...,Virginia Beach
18113,Secondi,/biz/secondi-washington?osq=Boutique+Shopping,Washington
18291,Lucinda’s,/biz/lucindas-wichita?osq=Boutique+Shopping,Wichita


# Finalise the data into tabs 

## Overall Cities

Overall Cities: isolate the top 10 cities with the highest average rating across independent stores.

In [731]:
overall_cities = weighted_avg_rating[weighted_avg_rating.stores_number>=31].sort_values(by='weighted_avg_rating', ascending=False).head(10)

In [732]:
overall_cities = overall_cities.merge(top_stores, on='City', how='left')

In [733]:
overall_cities

,City,State,weighted_avg_rating,stores_number,top_shop,shop_url
0,Corpus Christi,TX,4.395395,66,Island Surf Rentals,/biz/island-surf-rentals-port-aransas-2?osq=Bo...
1,San Francisco,CA,4.286984,234,On The Run,/biz/on-the-run-san-francisco?osq=Boutique+Sho...
2,Newark,NJ,4.283310,115,Facets,/biz/facets-brooklyn-2?osq=Boutique+Shopping
3,Los Angeles,CA,4.231633,237,Miss Lola,/biz/miss-lola-pico-rivera?osq=Boutique+Shopping
4,Stockton,CA,4.167298,149,Dhoom Indian Fashion Clothing and Bridal,/adredir?ad_business_id=VVzEB9U_0BDkVLaaANvfJQ...
5,Chicago,IL,4.151319,241,Bryn Mawr Jewelry Company,/biz/bryn-mawr-jewelry-company-chicago?osq=Bou...
6,Denver,CO,4.150038,228,Goodwill Archer Store,/biz/goodwill-archer-store-denver?osq=Boutique...
7,Portland,OR,4.119385,227,Adidas Employee Store,/biz/adidas-employee-store-portland-4?osq=Bout...
8,Scottsdale,AZ,4.115425,223,Uptown Cheapskate- Chandler,/biz/uptown-cheapskate-chandler-chandler-2?osq...
9,Aurora,CO,4.107941,230,d’Anelli Bridal,/biz/d-anelli-bridal-lakewood-2?osq=Boutique+S...


## Stores

Stores: isolate the top 10 independent stores across all cities with the highest avg rating.

In [734]:
stores = top_20[:11]

In [735]:
stores

,shop_name,shop_url,star_rating,reviews,category_1,category_2,category_3,City,State,weighted_rating
118,Happy Jewelers,/biz/happy-jewelers-fullerton-2?osq=Boutique+S...,4.9,1623,Jewelry,Watches,Jewelry Repair,Anaheim,CA,7952.7
15746,Trudys Brides & Special Occasions,/adredir?ad_business_id=TAzqX8RQr1GvPYnqgZwGZQ...,3.9,1815,Women's Clothing,Bridal,None,San Jose,CA,7078.5
16185,Tustin Jewelry Exchange - Direct Diamond Importer,/biz/tustin-jewelry-exchange-direct-diamond-im...,4.4,1440,Jewelry,Jewelry Repair,None,Santa Ana,CA,6336.0
15272,Road Runner Sports,/biz/road-runner-sports-san-diego?osq=Boutique...,3.7,1574,Sports Wear,Shoe Stores,None,San Diego,CA,5823.8
5242,Janene’s Bridal Boutique,/biz/janenes-bridal-boutique-alameda?osq=Bouti...,4.5,1099,Bridal,Accessories,Sewing & Alterations,Fremont,CA,4945.5
11923,Janene’s Bridal Boutique,/biz/janenes-bridal-boutique-alameda?osq=Bouti...,4.5,1097,Bridal,Accessories,Sewing & Alterations,Oakland,CA,4936.5
2939,The Diamond Connection,/biz/the-diamond-connection-san-diego-2?osq=Bo...,4.9,929,Jewelry,Diamond Buyers,None,Chula Vista,CA,4552.1
2954,David’s Bridal,/biz/davids-bridal-san-diego-5?osq=Boutique+Sh...,3.6,1210,Bridal,Accessories,Shoe Stores,Chula Vista,CA,4356.0
6630,Don Quijote - Kaheka,/biz/don-quijote-kaheka-honolulu-2?osq=Boutiqu...,4.0,1084,Grocery,Department Stores,International Grocery,Honolulu,HI,4336.0
5062,Sports Basement,/biz/sports-basement-sunnyvale-2?osq=Boutique+...,4.2,975,Sports Wear,Bikes,Outdoor Gear,Fremont,CA,4095.0


## Jewelry

Jewelry: isolate the top 10 cities with the highest average rating across independent stores in this category.

In [755]:
jewelry = weighted_avg_categories[weighted_avg_categories.category=='Jewelry']

In [756]:
jewelry.store_count.describe()

count    92.000000
mean     35.076087
std      14.332959
min      10.000000
25%      23.750000
50%      36.000000
75%      42.250000
max      75.000000
Name: store_count, dtype: float64

In [757]:
jewelry = jewelry[jewelry.store_count>=24]

In [758]:
jewelry = jewelry.sort_values(by='weighted_avg_rating', ascending=False).head(10)

In [759]:
jewelry = jewelry.merge(top_stores_jewelry, on='City', how='left')

In [760]:
jewelry

,City,State,category,weighted_avg_rating,store_count,top_shop,shop_url
0,Los Angeles,CA,Jewelry,4.639321,37,Artisan LA Jewelry,/biz/artisan-la-jewelry-los-angeles?osq=Boutiq...
1,Riverside,CA,Jewelry,4.623432,29,Desire Jewelry,/biz/desire-jewelry-glendora-2?osq=Boutique+Sh...
2,Indianapolis,IN,Jewelry,4.544402,43,Silver In the City,/biz/silver-in-the-city-indianapolis?osq=Bouti...
3,Fremont,CA,Jewelry,4.515622,44,Geoffrey’s Diamonds & Goldsmith,/biz/geoffreys-diamonds-and-goldsmith-san-carl...
4,Chula Vista,CA,Jewelry,4.515316,42,The Diamond Connection,/biz/the-diamond-connection-san-diego-2?osq=Bo...
5,Stockton,CA,Jewelry,4.502938,27,Varsha Patel,/adredir?ad_business_id=gkpk_aNfmA0d0ckuypUOiQ...
6,Buffalo,NY,Jewelry,4.497688,26,Barbara Oliver Jewelry,/biz/barbara-oliver-jewelry-williamsville-3?os...
7,Anaheim,CA,Jewelry,4.496353,39,Happy Jewelers,/biz/happy-jewelers-fullerton-2?osq=Boutique+S...
8,Portland,OR,Jewelry,4.495114,60,Medallion Jewelers,/biz/medallion-jewelers-portland-2?osq=Boutiqu...
9,Atlanta,GA,Jewelry,4.494432,41,Worthmore Jewelers,/biz/worthmore-jewelers-decatur?osq=Boutique+S...


## Accessories

Accessories: isolate the top 10 cities with the highest average rating across independent stores in this category.

In [761]:
accessories = weighted_avg_categories[weighted_avg_categories.category=='Accessories']

accessories.store_count.describe()

count     99.000000
mean      77.525253
std       33.070840
min       11.000000
25%       43.000000
50%       89.000000
75%      101.500000
max      140.000000
Name: store_count, dtype: float64

In [762]:
accessories = accessories[accessories.store_count>=43]

In [763]:
accessories = accessories.sort_values(by='weighted_avg_rating', ascending=False).head(10)

accessories = accessories.merge(top_stores_accessories, on='City', how='left')

accessories

,City,State,category,weighted_avg_rating,store_count,top_shop,shop_url
0,Stockton,CA,Accessories,4.326658,54,Le Caprice,/biz/le-caprice-cupertino-2?osq=Boutique+Shopping
1,Oakland,CA,Accessories,4.302403,93,Janene’s Bridal Boutique,/biz/janenes-bridal-boutique-alameda?osq=Bouti...
2,Phoenix,AZ,Accessories,4.279384,95,Nordstrom Rack - E Camelback Rd,/adredir?ad_business_id=5lArDVQZBp97qykssYsVGA...
3,Los Angeles,CA,Accessories,4.274907,130,House Of CB,/biz/house-of-cb-west-hollywood?osq=Boutique+S...
4,Scottsdale,AZ,Accessories,4.263301,98,Bunky Boutique,/biz/bunky-boutique-phoenix?osq=Boutique+Shopping
5,Mesa,AZ,Accessories,4.258376,90,Lizard Thicket,/biz/lizard-thicket-mesa?osq=Boutique+Shopping
6,Virginia Beach,VA,Accessories,4.240909,63,Anderson’s - Newport News,/biz/andersons-newport-news-newport-news?osq=B...
7,North Las Vegas,NV,Accessories,4.223177,87,Mocha & Karamelo,/biz/mocha-and-karamelo-las-vegas?osq=Boutique...
8,Portland,OR,Accessories,4.209102,100,Adidas Employee Store,/biz/adidas-employee-store-portland-4?osq=Bout...
9,San Diego,CA,Accessories,4.190406,104,Pigment,/biz/pigment-san-diego?osq=Boutique+Shopping


## Women's Clothing

In [764]:
women_clothing = weighted_avg_categories[weighted_avg_categories.category=='Women\'s Clothing']

women_clothing.store_count.describe()

count    100.000000
mean     105.810000
std       45.489636
min       13.000000
25%       59.500000
50%      125.000000
75%      139.250000
max      187.000000
Name: store_count, dtype: float64

In [765]:
women_clothing = women_clothing[women_clothing.store_count>=60]

In [766]:
women_clothing = women_clothing.sort_values(by='weighted_avg_rating', ascending=False).head(10)

women_clothing = women_clothing.merge(top_stores_women_clothing, on='City', how='left')

women_clothing

,City,State,category,weighted_avg_rating,store_count,top_shop,shop_url
0,Chesapeake,VA,Women's Clothing,4.344792,86,Uptown Cheapskate - Virginia Beach,/biz/uptown-cheapskate-virginia-beach-virginia...
1,Virginia Beach,VA,Women's Clothing,4.234532,85,Uptown Cheapskate - Virginia Beach,/biz/uptown-cheapskate-virginia-beach-virginia...
2,Los Angeles,CA,Women's Clothing,4.209042,182,Miss Lola,/biz/miss-lola-pico-rivera?osq=Boutique+Shopping
3,Tampa,FL,Women's Clothing,4.195173,159,Uptown Cheapskate,/biz/uptown-cheapskate-tampa?osq=Boutique+Shop...
4,Stockton,CA,Women's Clothing,4.189474,79,Dhoom Indian Fashion Clothing and Bridal,/adredir?ad_business_id=VVzEB9U_0BDkVLaaANvfJQ...
5,San Francisco,CA,Women's Clothing,4.182694,110,Akarshan Designs Boutique,/biz/akarshan-designs-boutique-san-bruno?osq=B...
6,St. Petersburg,FL,Women's Clothing,4.155382,149,Uptown Cheapskate,/biz/uptown-cheapskate-tampa?osq=Boutique+Shop...
7,Louisville,KY,Women's Clothing,4.130828,99,Work the Metal,/biz/work-the-metal-louisville?osq=Boutique+Sh...
8,Aurora,CO,Women's Clothing,4.093211,141,The Barn Antiques & Specialty Shops,/biz/the-barn-antiques-and-specialty-shops-cas...
9,Denver,CO,Women's Clothing,4.088795,137,SOL,/biz/sol-denver?osq=Boutique+Shopping


# Create an Excel file

In [767]:
# Create an Excel writer object

writer = pd.ExcelWriter('The Cities With the Most Loved Independent Stores.xlsx')

In [768]:
overall_cities.to_excel(writer, sheet_name='Overall Cities', index=False)
stores.to_excel(writer, sheet_name='Stores', index=False)
jewelry.to_excel(writer, sheet_name='Jewelry', index=False)
accessories.to_excel(writer, sheet_name='Accessories', index=False)
women_clothing.to_excel(writer, sheet_name='Women\'s Clothing', index=False)
data.to_excel(writer, sheet_name='Raw Data', index=False)

In [769]:
# Save the Excel file
writer.save()